In [1]:
import sys

sys.path.append("/Users/j/Documents/Forme/code/forme-groups")

from src.groups.unit import UnitType, UnitTypeRef
from src.groups.group import Group


def main():
    # Create a default group
    group = Group(
        system_types_path="/Users/j/Documents/Forme/code/forme-groups/src/groups/system_types.json"
    )

    # Create a new unit type
    new_unit_type = UnitType(
        aliases= (UnitTypeRef("test_unit_type"),),
        super_type=UnitTypeRef("str"),
        prefix="test<",
        suffix=">",
        separator="|",
        sys_function=None
    )

    # Add a unit type to the group
    group._group_unit_generator.unit_generator.unit_type_pool.add_unit_type(
        new_unit_type
    )

    # freeze the unit types
    group.freeze_unit_types()

    print(group._group_unit_generator.unit_generator.unit_type_pool.get_type_aliases())

    print(group.group_units)
    
    unit1 = group.create_group_unit()

    group.create_group_unit(active_unit = unit1)

    print(group.group_units)



main()

TypeError: UnitTypePool.add_unit_type() takes 2 positional arguments but 3 were given